In [120]:
from glob import glob
from os.path import isdir
import os
import pandas as pd
abnormal=[]
normal=[]
for x in glob('../data/*/*걷기 무게중심*'):
    abnormal.append(x)
for x in glob('../data/*/*걷기_보통*'):
    normal.append(x)

In [121]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [122]:
scale_columns=['x','y','z']
print(len(abnormal))
print(len(normal))

40
10


In [76]:
df=pd.read_excel(open(normal[0], 'rb'))
df.head()

,Unnamed: 0,시간(ms),이름,허리인치,키,회차,낙상종류,방향,x,y,z
0,0,70,고대석,72kg,168,1,걷기,보통,644,15552,7488
1,1,127,고대석,72kg,168,1,걷기,보통,-3835,11648,6848
2,2,164,고대석,72kg,168,1,걷기,보통,-3259,15040,3968
3,3,202,고대석,72kg,168,1,걷기,보통,-123,16256,3904
4,4,258,고대석,72kg,168,1,걷기,보통,1604,12160,2816


In [123]:
X_train_df=pd.DataFrame()
X_test_df=pd.DataFrame()
y_train=[]
y_test=[]
for f in abnormal:
    df=pd.read_excel(open(f, 'rb'))  
    X_train_df=X_train_df.append(df.loc[:1112,scale_columns],ignore_index=True)
    X_test_df=X_test_df.append(df.loc[1112:1391,scale_columns])
for f in normal:
    df=pd.read_excel(open(f, 'rb'))  
    X_train_df=X_train_df.append(df.loc[:1112,scale_columns])
    X_test_df=X_test_df.append(df.loc[1112:1391,scale_columns])


### X_train, X_test (44520:11131 / 11199: 2800)

In [48]:
X_train_df.head()

,x,y,z
0,1330,15488,6272
1,1842,12480,4608
2,-1293,11264,3712
3,-1805,13760,3520
4,-2317,14464,3648


In [49]:
X_test_df.head()

,x,y,z
1112,-2061,17856,832
1113,50,18752,6400
1114,-1101,17024,8064
1115,9266,10240,7168
1116,-461,14656,7424


### y_train, y_test ( 0: abnormal, 1: normal)

In [124]:
y_train=[]
y_test=[]
for i in range(55650):
    if i <=44520:
        y_train.append(0) #비정상
    else:
        y_train.append(1) #정상
for i in range(13999):
    if i <=11199:
        y_test.append(0) #비정상
    else:
        y_test.append(1) #정상

In [125]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

scaler = scaler.fit(X_train_df)

X_train_df = scaler.transform(
  X_train_df.to_numpy()
)

X_test_df= scaler.transform(
  X_test_df.to_numpy()
)

In [52]:
X_train_df.shape

(55650, 3)

In [126]:
import numpy as np
y_train=np.array(y_train).reshape(-1,1)
y_train.shape

(55650, 1)

In [127]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X[i:(i + time_steps)]
        labels = y[i + time_steps]
        Xs.append(v)
        ys.append(labels)
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [128]:
TIME_STEPS = 233
STEP = 40

X_train, y_train = create_dataset(
    X_train_df,
    y_train,
    TIME_STEPS,
    STEP
)

X_test, y_test = create_dataset(
    X_test_df,
    y_test,
    TIME_STEPS,
    STEP
)

In [56]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1386, 233, 3)
(1386, 1)
(345, 233, 3)
(345, 1)


In [129]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

In [144]:
from keras.layers import LSTM
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=64,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

In [148]:

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from datetime import datetime

In [152]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]))
model.add(LSTM(50, return_sequences= False))
model.add(Dense(25))
model.add(Dense(1))
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])
model.summary()
# h

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 233, 50)           10800     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 26        
Total params: 32,301
Trainable params: 32,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

In [146]:
y_pred=model.predict(X_test)

In [ ]:
model.evaluate(X_test, y_test)

#### To Do - hyperparameter tuning

In [62]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import matplotlib.pyplot as plt

In [141]:
from keras.layers import LSTM
from keras.layers import Dropout
model = Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    ))
model.add(keras.layers.Dropout(0.2))


model.add(keras.layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


In [113]:
DATA.head()

""


In [ ]:
 def create_dataset(X, y, time_steps=1, step=1):
        Xs, ys = [], []
        for i in range(0, len(X) - time_steps, step):
            v = X[i:(i + time_steps)]
            labels = y[i + time_steps]
            Xs.append(v)
            ys.append(labels)
        return np.array(Xs), np.array(ys).reshape(-1, 1)
create_dataset(DATA['x','y','z'],DATA['낙상종류'])

In [69]:
import pandas as pd
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import *

class TimeBasedCV(object):
    '''
    Parameters 
    ----------
    train_period: int
        number of time units to include in each train set
        default is 30
    test_period: int
        number of time units to include in each test set
        default is 7
    freq: string
        frequency of input parameters. possible values are: days, months, years, weeks, hours, minutes, seconds
        possible values designed to be used by dateutil.relativedelta class
        deafault is days
    '''
    
    
    def __init__(self, train_period=30, test_period=7, freq='seconds'):
        self.train_period = train_period
        self.test_period = test_period
        self.freq = freq

        
        
    def split(self, data, validation_split_date=None, date_column='시간(ms)', gap=0):
        '''
        Generate indices to split data into training and test set
        
        Parameters 
        ----------
        data: pandas DataFrame
            your data, contain one column for the record date 
        validation_split_date: datetime.date()
            first date to perform the splitting on.
            if not provided will set to be the minimum date in the data after the first training set
        date_column: string, deafult='record_date'
            date of each record
        gap: int, default=0
            for cases the test set does not come right after the train set,
            *gap* days are left between train and test sets
        
        Returns 
        -------
        train_index ,test_index: 
            list of tuples (train index, test index) similar to sklearn model selection
        '''
        
        # check that date_column exist in the data:
        try:
            data[date_column]
        except:
            raise KeyError(date_column)
                    
       
        
        
    def create_dataset(X, y, time_steps=1, step=1):
        Xs, ys = [], []
        for i in range(0, len(X) - time_steps, step):
            v = X[i:(i + time_steps)]
            labels = y[i + time_steps]
            Xs.append(v)
            ys.append(labels)
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    
    def get_n_splits(self):
        """Returns the number of splitting iterations in the cross-validator
        Returns
        -------
        n_splits : int
            Returns the number of splitting iterations in the cross-validator.
        """
        return self.n_splits 

In [ ]:
tscv = TimeBasedCV(train_period=30,
                   test_period=7,
                   freq='days')


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)